In [1]:
import os
import getpass
from uuid import uuid4

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE5 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

In [3]:
from langchain_community.document_loaders import DirectoryLoader

In [9]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [5]:
!pip install libmagic

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for libmagic: filename=libmagic-1.0-py3-none-any.whl size=4325 sha256=0e6d984a614e8d9f5030da28214772321883e0e745aaf162fd2232f65bb9dab5
  Stored in directory: c:\users\dabra\appdata\local\pip\cache\wheels\dd\06\e7\972d8dfb613a9a618e0c2314d63c1b625a51cdb7af785811dd
Successfully built libmagic



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [10]:
import sys
print(sys.executable)
!{sys.executable} -m pip install libmagic

c:\Users\dabra\AppData\Local\Programs\Python\Python39\python.exe


You should consider upgrading via the 'c:\Users\dabra\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [ ]:
from langgraph.prebuilt import ToolExecutorAgent
from langchain.tools.retriever import create_retriever_tool

In [14]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5) # this tool enables users to retrieve up-to-date online information directly within the Langchain framework

tool_belt = [
    tavily_tool, # tool to  facilitate real-time web searches by leveraging the Tavily API 
    ArxivQueryRun(), # tool to facilitate the execution of queries within the Langchain framework
]

In [15]:
model = model.bind_tools(tool_belt) 

In [20]:
rag_documents = docs

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [24]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [ ]:
# Wrap retriever as a tool
retriever_tool = create_retriever_tool(retriever, name="retriever", description="Fetch relevant documents")

# Define the LangGraph agent
llm = ChatOpenAI(model="gpt-4")

agent = ToolExecutorAgent(
    llm=llm,
    tools=[retriever_tool]
)

In [16]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

NameError: name 'vectorstore' is not defined

In [13]:
from langgraph.prebuilt import ToolExecutorAgent
from langchain.chat_models import ChatOpenAI
from langchain.tools.retriever import create_retriever_tool

ImportError: cannot import name 'ToolExecutorAgent' from 'langgraph.prebuilt' (c:\Users\dabra\AppData\Local\Programs\Python\Python39\lib\site-packages\langgraph\prebuilt\__init__.py)

In [ ]:
# Define your retriever
retriever = some_vectorstore.as_retriever()

# Wrap retriever as a tool
retriever_tool = create_retriever_tool(retriever, name="retriever", description="Fetch relevant documents")

# Define the LangGraph agent
llm = ChatOpenAI(model="gpt-4")

agent = ToolExecutorAgent(
    llm=llm,
    tools=[retriever_tool]
)
